In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the CSV file (test dataset)
test_data = pd.read_csv("test_dataset.csv")

# Load model and tokenizer
model_name = "distilbert_fake_news_classification"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


model.eval()

# Tokenize the test dataset
inputs = tokenizer(
    list(test_data['text']),
    padding=True,
    truncation=True,
    return_tensors="pt",
    max_length=128
)


predictions = []


with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

    predicted_classes = torch.argmax(logits, dim=-1)  # Get class indices
    predictions = predicted_classes.cpu().numpy()  # Convert to numpy array for easy use

# Add the predictions to the test dataset
test_data['predictions'] = predictions

test_data.to_csv("test_predictions.csv", index=False)
